**Author**: Todd Goldfarb (AI Dev)

**Contact**: tcgoldfarb@gmail.com

**Date**: 7/26/2023

This is a **METAR-PREDICTIVE-KJFK-1DAY** model for the JFK Airport (ICAO CODE KJFK). From what I believe, an RNN network would be ideal for this first experiment (LSTM specific).  We will have to pre-process and create our own dataset.

Using .TXT and .CSV files of METAR data related to KJFK, we will isolate the viable inputs and space the answer input by a distance of one day (from the latest METAR given). For example, if we give three months worth of METAR data related to KJFK, we will space it one day off of the lastest day in the three months of data. We will be using 7+ total months of KJFK METAR data.

Ideally, the model takes in about a week's worth of input data, and can give the predictive following day.

(link to the one of the METAR sources, stores historical data but is in .TXT format: http://www.ogimet.com/metars.phtml.en)
- This link only accepts 31 days at a time, but we can take multiple chunks. I've taken 7 chunks and put them together on a txt file in my github public data repo.

(link to a dataserver of CSV data, not airport specific, only stores 3 days of data then clears: https://www.aviationweather.gov/dataserver)
- **Will be great for testing!**

(link to my public Github repo, which has all of the data used for models: https://github.com/Todd-C-Goldfarb/dataForModels)

****

In [ ]:
# GET THE DATA FROM THE PUBLIC GITHUB REPO

!wget https://raw.githubusercontent.com/Todd-C-Goldfarb/dataForModels/main/JFK_DATA.txt
# https://github.com/Todd-C-Goldfarb/dataForModels/blob/main/KJFK_METAR/KJFK_METAR_TOTAL_01_26_2023-07_26_2023.txt

--2023-09-21 19:28:31--  https://raw.githubusercontent.com/Todd-C-Goldfarb/dataForModels/main/JFK_DATA.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1162690 (1.1M) [text/plain]
Saving to: ‘JFK_DATA.txt’

JFK_DATA.txt        100%[===================>]   1.11M  --.-KB/s    in 0.08s   

2023-09-21 19:28:32 (14.3 MB/s) - ‘JFK_DATA.txt’ saved [1162690/1162690]



****
Now that we have a .TXT file filled with lines of METAR data (going back to 2006), I will explain the logistics of a METAR below:

**Type of report**: METAR or SPECI. METAR is a routine weather report, issued every hour. SPECI is a special weather report issued when there are significant changes in weather conditions.
- **As the data has both SPECI and METAR data, we will have to process out all of the SPECI lines.**

**Station identifier**: A four-letter code as established by the International Civil Aviation Organization (ICAO).
- **Since the data only has KJFK as the ICAO, we can ignore this part of the data.**

Date and time: The day of the month and the time (in UTC) when the observation was made.
- **The first data point we need, as daytime is significant in relation to weather.**

Wind: Wind direction and speed. The first three digits indicate the direction the wind is coming from (in degrees). The last two or three digits indicate the speed of the wind in knots.
- **Another important datapoint, knowing wind direction and speed is significant for knowing which runway to land on, and how to land to account for wind.**

Visibility: Reported in meters or statute miles.
- **Another important datapoint, VFR or IFR conditions, etc.**

Weather phenomena: Descriptions of significant weather phenomena.
- **We will need to modify some of the letters into token numbers as representations**

Sky condition: Amount and type of cloud cover.
- **Same issue with weather phenomena, we will have to turn into token numbers.**

Temperature and dew point: In degrees Celsius.
- **Another important datapoint.**

Sea level pressure: In inches of mercury (in the U.S.) or hectopascals/millibars.
- **Related to temp and actual altitude of ground level.**

Remarks: Additional information.
- **We will ignore remarks, as they are super conditional. We couldn't possibly generate them in advance.**

In [ ]:
# Now that we have the huge METAR file, we need to turn it into a PANDAS dataframe.

!pip install metar

from metar import Metar
# WE WILL USE THE METAR library, VERY HELPFUL!

import pandas as pd

def flip_and_prep_data(filename):
    with open(filename, 'r') as f:
        lines = f.readlines()

    metar_lines = []
    for line in lines[::-1]:  # BACK TO FRONT
        metar_lines.append(line[18:].strip())

    return metar_lines

def parse_metar(lines):
    data = []
    for line in lines:
        try:
            # Get rid of NIL lines
            if "NIL" in line:
              continue
            # strip the first timestamp and only retain the METAR string
            line = ' '.join(line.split()[1:])

            # For some reason, metar library hates the 31st day of the month
            if line[11:13] == '31':
                continue

            # Catch a possible METAR typo (/010 Celsius 0 typo)
            line = line.replace("/010", "/10")
            # line = line.split('T0')[0]

            #COR is a CORRECTION TAG, which is throwing off the Parser
            line = line.replace("COR ", "")

            # Remove any SPECI lines
            if "SPECI" in line:
              continue


            obs = Metar.Metar(line)

            if obs.temp:
              temperature = obs.temp.string("C")
            else:
              temperature = None
            if obs.dewpt:
              dew_point = obs.dewpt.string("C")
            else:
              dew_point = None
            if obs.press:
              pressure = obs.press.string("mb")
            else:
              pressure = None
            metar_dict = {
                'datetime': obs.time,
                'wind': obs.wind(),
                'visibility': obs.visibility(),
                'temperature': obs.temp.string("C"),
                'dew_point': obs.dewpt.string("C"),
                'pressure': obs.press.string("mb"),
            }
            data.append(metar_dict)
        except Exception as e:
           # print(f"Error parsing line: {line}. Error: {e}")
           continue
    return pd.DataFrame(data)

lines = flip_and_prep_data('JFK_DATA.txt')
print(lines)
df = parse_metar(lines)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.4/201.4 kB 4.0 MB/s eta 0:00:00
['METAR KJFK 010051Z 09006KT 10SM FEW250 11/02 A3037 RMK AO2 SLP284 T01060022=', 'METAR KJFK 010151Z 00000KT 10SM CLR 11/03 A3039 RMK AO2 SLP290 T01060033=', 'METAR KJFK 010251Z 08003KT 10SM CLR 09/03 A3038 RMK AO2 SLP288 T00890028 50008=', 'METAR KJFK 010351Z 36005KT 10SM CLR 11/02 A3037 RMK AO2 SLP284 T01060022=', 'METAR KJFK 010451Z 03006KT 10SM CLR 09/03 A3037 RMK AO2 SLP284 T00940028 402110056=', 'METAR KJFK 010551Z 36007KT 10SM CLR 08/02 A3036 RMK AO2 SLP280 T00830022 10117 20083 58008=', 'METAR KJFK 010651Z 01008KT 10SM CLR 09/01 A3034 RMK AO2 SLP275 T00940011=', 'METAR KJFK 010751Z 03008KT 10SM CLR 09/M02 A3035 RMK AO2 SLP275 T00941022=', 'METAR KJFK 010851Z 04009KT 10SM CLR 09/M04 A3035 RMK AO2 SLP276 T00891039 55005=', 'METAR KJFK 010951Z 03010KT 10SM CLR 10/M05 A3035 RMK AO2 SLP276 T01001050=', 'METAR KJFK 011051Z 03011KT 10SM FEW060 11/M04 A3036 RMK AO2 SLP279 T01061044=', 'METAR KJFK 011151Z 0

In [ ]:
# Let's take a look at our df as it is now
df

# It's decent, but the 'sentence' structure of the wind column won't be able to be eaten by the model.
# I'm going to seperate the wind into two more columns, direction and strength.
# As well, we need to remove the units so that it's pure numbers in each section.

datetime            wind visibility temperature dew_point  \
0     2023-09-01 00:51:00    E at 6 knots   10 miles      10.6 C     2.2 C   
1     2023-09-01 01:51:00            calm   10 miles      10.6 C     3.3 C   
2     2023-09-01 02:51:00    E at 3 knots   10 miles       8.9 C     2.8 C   
3     2023-09-01 03:51:00    N at 5 knots   10 miles      10.6 C     2.2 C   
4     2023-09-01 04:51:00  NNE at 6 knots   10 miles       9.4 C     2.8 C   
...                   ...             ...        ...         ...       ...   
10301 2023-09-02 04:51:00  SSW at 7 knots   10 miles      16.7 C    12.2 C   
10302 2023-09-02 05:51:00  WSW at 7 knots   10 miles      16.7 C    13.3 C   
10303 2023-09-02 06:51:00    W at 8 knots   10 miles      17.2 C    13.9 C   
10304 2023-09-02 07:51:00    W at 9 knots   10 miles      16.7 C    12.8 C   
10305 2023-09-02 08:51:00  WNW at 6 knots   10 miles      16.1 C    12.2 C   

        pressure  
0      1028.4 mb  
1      1029.1 mb  
2      1028.8 mb  
3      1028.4 mb  
4      1028.4 mb  
...          ...  
10301  1025.4 mb  
10302  1025.1 mb  
10303  1024.7 mb  
10304  1024.4 mb  
10305  1024.7 mb  

[10306 rows x 6 columns]

In [ ]:
# FURTHER PROCESSING THE DATASET

df['visibility'] = df['visibility'].astype(str)
df['visibility'] = df['visibility'].str.extract('(\d+)').astype(float)

df['temperature'] = df['temperature'].astype(str)
df['temperature'] = df['temperature'].str.replace(' C', '').astype(float)

df['dew_point'] = df['dew_point'].astype(str)
df['dew_point'] = df['dew_point'].str.replace(' C', '').astype(float)

df['pressure'] = df['pressure'].astype(str)
df['pressure'] = df['pressure'].str.replace(' mb', '').astype(float)

df[['wind_direction', 'wind_speed']] = df['wind'].str.extract('(\w+) at (\d+)')

df['wind_speed'] = pd.to_numeric(df['wind_speed'])

direction_dict = {'N': 0, 'NNE': 22.5, 'NE': 45, 'ENE': 67.5, 'E': 90, 'ESE': 112.5,
                  'SE': 135, 'SSE': 157.5, 'S': 180, 'SSW': 202.5, 'SW': 225, 'WSW': 247.5,
                  'W': 270, 'WNW': 292.5, 'NW': 315, 'NNW': 337.5}

df['wind_direction'] = df['wind_direction'].map(direction_dict)

df = df.drop('wind', axis=1)

df['hour'] = df['datetime'].dt.hour

# WE PROBABLY DON'T NEED THE DATETIME, OR HOUR.
# DELETE LINE BELOW IF NEEDED AT SOME POINT.
df = df.drop('datetime', axis=1)
df = df.drop('hour', axis=1)

df

visibility  temperature  dew_point  pressure  wind_direction  \
0            10.0         10.6        2.2    1028.4            90.0   
1            10.0         10.6        3.3    1029.1             NaN   
2            10.0          8.9        2.8    1028.8            90.0   
3            10.0         10.6        2.2    1028.4             0.0   
4            10.0          9.4        2.8    1028.4            22.5   
...           ...          ...        ...       ...             ...   
10301        10.0         16.7       12.2    1025.4           202.5   
10302        10.0         16.7       13.3    1025.1           247.5   
10303        10.0         17.2       13.9    1024.7           270.0   
10304        10.0         16.7       12.8    1024.4           270.0   
10305        10.0         16.1       12.2    1024.7           292.5   

       wind_speed  
0             6.0  
1             NaN  
2             3.0  
3             5.0  
4             6.0  
...           ...  
10301         7.0  
10302         7.0  
10303         8.0  
10304         9.0  
10305         6.0  

[10306 rows x 6 columns]

Now that we have our dataset looking great, we need to start setting up our model and training.

Need to normalize the data and split the dataset.

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# GET RID OF ALL NANs!
df.fillna(0, inplace=True)

# USING MINMAX
scaler = MinMaxScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

# 80% SPLIT IS STANDARD
train_size = int(len(df_scaled) * 0.8)
train, test = df_scaled.iloc[:train_size], df_scaled.iloc[train_size:]

# Func for creating the sequences
# we have 7*24 = 168 inputs and 1 output
def create_sequences(data, seq_length):
    xs = []
    ys = []

    for i in range(len(data)-seq_length):
        x = data.iloc[i:(i+seq_length)].values
        y = data.iloc[i+seq_length].values
        xs.append(x)
        ys.append(y)

    return np.array(xs), np.array(ys)

SEQ_LENGTH = 168
x_train, y_train = create_sequences(train, SEQ_LENGTH)
x_test, y_test = create_sequences(test, SEQ_LENGTH)

The model's definition comes next, decided to use tf and keras.

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

model = Sequential([
    # LSTM LAYER
    LSTM(256, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])),
    # PLAY WITH DROPOUTS, PREVENT OVERFITTING
    Dropout(0.2),
    # ANOTHER 64 LAYERS, NO RETURN
    LSTM(128, return_sequences=False),
    # ANOTHER DROPOUT, JUST FOR FUN
    Dropout(0.2),
    Dense(6)  # 6 CATEGORIES FOR OUT PUT
])

model.compile(optimizer='adam', loss='mse')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 168, 256)          269312    
                                                                 
 dropout (Dropout)           (None, 168, 256)          0         
                                                                 
 lstm_1 (LSTM)               (None, 128)               197120    
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense (Dense)               (None, 6)                 774       
                                                                 
Total params: 467206 (1.78 MB)
Trainable params: 467206 (1.78 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


TIME TO TRAIN!

In [ ]:
trained = model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))

Epoch 1/10
253/253 [==============================] - 232s 879ms/step - loss: 0.0183 - val_loss: 0.0104
Epoch 2/10
253/253 [==============================] - 220s 870ms/step - loss: 0.0122 - val_loss: 0.0092
Epoch 3/10
253/253 [==============================] - 220s 868ms/step - loss: 0.0112 - val_loss: 0.0091
Epoch 4/10
253/253 [==============================] - 220s 870ms/step - loss: 0.0106 - val_loss: 0.0088
Epoch 5/10
253/253 [==============================] - 220s 869ms/step - loss: 0.0103 - val_loss: 0.0088
Epoch 6/10
253/253 [==============================] - 225s 890ms/step - loss: 0.0101 - val_loss: 0.0088
Epoch 7/10
253/253 [==============================] - 219s 868ms/step - loss: 0.0098 - val_loss: 0.0087
Epoch 8/10
253/253 [==============================] - 219s 866ms/step - loss: 0.0098 - val_loss: 0.0085
Epoch 9/10
253/253 [==============================] - 220s 870ms/step - loss: 0.0096 - val_loss: 0.0086
Epoch 10/10
253/253 [==============================] - 229s 904m

Let's test if it works.

In [ ]:
test_loss = model.evaluate(x_test, y_test, verbose=1)
print(f'Test Loss: {test_loss}')

60/60 [==============================] - 15s 250ms/step - loss: 0.0084
Test Loss: 0.008359738625586033


In [ ]:
# EXAMPLE GENERATION/PREDICTION
import math
# NEED TO PROCESS SOME INPUT DATA
sample_index = np.random.randint(0, len(df) - SEQ_LENGTH)
sample = df.iloc[sample_index:sample_index+SEQ_LENGTH]
goal = df.iloc[sample_index+SEQ_LENGTH]
sample_scaled = scaler.transform(sample)
sample_scaled = np.array([sample_scaled])

prediction = model.predict(sample_scaled)

prediction_original_scale = scaler.inverse_transform(prediction)

def format_as_metar(prediction):
    # EXACT CATEGORIES NEED TO BE TESTED
    visibility = math.ceil(prediction[0])
    temperature = prediction[1]
    dew_point = prediction[2]
    pressure = prediction[3]
    wind_dir = prediction[4]
    wind_speed = prediction[5]

    # FORMAT AS A STANDARD METAR
    # INSTEAD OF METAR AUTO, WE CAN MAKE A NEW ONE! METAR AI OR METAR PREDICT
    return f'VIS {visibility:.1f}, TEMP{temperature:.1f}, DEW{dew_point:.1f}, PRES {pressure:.1f}, WIND_DIR {wind_dir:.1f}, WIND_SP {wind_speed:.1f}'
    # return f'METAR AI {temperature:.1f}/{dew_point:.1f}KT {wind_speed:.1f}'

print("THE METAR DATA FED INTO MODEL:")
print(sample)
print("--------------------------------")
print("THE FOLLOWING PREDICTED METAR:")
metar_string = format_as_metar(prediction_original_scale[0])
print(metar_string)
print("THE CONTROL (ACTUAL) METAR:")
print(goal)

1/1 [==============================] - 1s 855ms/step
THE METAR DATA FED INTO MODEL:
      visibility  temperature  dew_point  pressure  wind_direction  wind_speed
970         10.0          5.6       -7.2    1018.3           292.5        13.0
971         10.0          6.7       -7.8    1018.0           292.5        12.0
972         10.0          8.3       -8.3    1016.9           270.0        14.0
973         10.0          8.9       -8.9    1016.9           292.5        10.0
974         10.0         10.0       -8.9    1016.3           292.5        12.0
...          ...          ...        ...       ...             ...         ...
1133        10.0          6.1        2.8     996.3             0.0        15.0
1134        10.0          6.1        2.8     997.6             0.0        15.0
1135        10.0          6.7        2.2     998.3             0.0        16.0
1136        10.0          6.7        1.1     998.6           337.5        15.0
1137        10.0          7.8        1.1     99

Now let's statistically see how accurate the model is, using the MEAN ABSOLUTE ERROR

In [ ]:
from sklearn.metrics import mean_absolute_error

predicted_values = [
    prediction_original_scale[0][0],
    prediction_original_scale[0][1],
    prediction_original_scale[0][2],
    prediction_original_scale[0][3],
    prediction_original_scale[0][4],
    prediction_original_scale[0][5]
]

actual_values = [
    goal["visibility"],
    goal["temperature"],
    goal["dew_point"],
    goal["pressure"],
    goal["wind_direction"],
    goal["wind_speed"]
]

# MAE
errors = [mean_absolute_error([predicted], [actual]) for predicted, actual in zip(predicted_values, actual_values)]


average_mae = sum(errors) / len(errors)

print(f"Average MAE: {average_mae}")
print(f"Estimated Accuracy: {100 - average_mae} %")


Average MAE: 39.11107389132182
Estimated Accuracy: 60.88892610867818 %


In [ ]:
#Generate 24 times for a full day
def dayGeneration():
  # Grab a random week to input
  # THIS IS NOT WORKING
  sample_index = np.random.randint(0, len(df) - SEQ_LENGTH)
  sample = df.iloc[sample_index:sample_index+SEQ_LENGTH]
  goal = df.iloc[sample_index+SEQ_LENGTH]
  sample_scaled = scaler.transform(sample)
  sample_scaled = np.array([sample_scaled])

  prediction = model.predict(sample_scaled)

  prediction_original_scale = scaler.inverse_transform(prediction)

  # We have our prediction, make the 23 loop.
  for i in range(23):
    sample = sample[1:] + prediction_original_scale
    #Indent one step, feed again.
    sample_scaled = scaler.transform(sample)
    sample_scaled = np.array([sample_scaled])
    goal = df.iloc[sample_index+SEQ_LENGTH+i]
    prediction = model.predict(sample_scaled)
    prediction_original_scale = scaler.inverse_transform(prediction)
    print("THE FOLLOWING PREDICTED METAR:")
    metar_string = format_as_metar(prediction_original_scale[0])
    print(metar_string)
    print("THE CONTROL (ACTUAL) METAR:")
    print(goal)


dayGeneration()


1/1 [==============================] - 2s 2s/step
THE FOLLOWING PREDICTED METAR:
VIS 37.0, TEMP3.8, DEW-17.1, PRES 1283.2, WIND_DIR 37.1, WIND_SP 2.2
THE CONTROL (ACTUAL) METAR:
visibility          10.0
temperature         21.1
dew_point            7.8
pressure          1011.5
wind_direction     337.5
wind_speed          16.0
Name: 8175, dtype: float64
1/1 [==============================] - 0s 71ms/step
THE FOLLOWING PREDICTED METAR:
VIS 44.0, TEMP18.9, DEW-7.6, PRES 1297.3, WIND_DIR 42.5, WIND_SP 8.8
THE CONTROL (ACTUAL) METAR:
visibility          10.0
temperature         22.2
dew_point            7.8
pressure          1010.8
wind_direction       0.0
wind_speed          19.0
Name: 8176, dtype: float64
1/1 [==============================] - 0s 71ms/step
THE FOLLOWING PREDICTED METAR:
VIS 46.0, TEMP26.9, DEW-2.4, PRES 1292.8, WIND_DIR 61.8, WIND_SP 9.9
THE CONTROL (ACTUAL) METAR:
visibility          10.0
temperature         23.9
dew_point            7.8
pressure          1010.2
wind_dir

Note: The internship has ended, and my work on this has stopped. The final steps would be to optimize the model and see if it could predict further, also needs a little bit of debug.